In [1]:
# !pip install matplotlib

In [2]:
import pandas as pd
import psycopg2
import os
import matplotlib as plt

# teste com um único arquivo

In [3]:
df = pd.read_parquet("./data/yellow_tripdata_2023-02.parquet")

In [4]:
df.shape

(2913955, 19)

In [5]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1.0,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142.0,163.0,2.0,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2.0,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71.0,71.0,4.0,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2.0,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71.0,71.0,4.0,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1.0,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132.0,26.0,1.0,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2.0,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161.0,145.0,1.0,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [6]:
df.tpep_pickup_datetime.min() # algum dado parece ter enviado errado pela empresa de taxi. Deveria contemplar só 2023-02

Timestamp('2008-12-31 23:05:06')

In [7]:
df.tpep_pickup_datetime.max()

Timestamp('2023-03-07 13:01:28')

In [8]:
CONNECTION = "<URL DA CONEXÃO>"
conn = psycopg2.connect(CONNECTION)
cursor = conn.cursor()

In [9]:
df.iloc[1]["VendorID"]

2.0

In [10]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')

In [11]:
# ver quantas corridas por dia e por hora existem
df['tpep_pickup_day'] = pd.to_datetime(df['tpep_pickup_datetime']).dt.strftime('%Y-%m-%d')
df['tpep_pickup_hour'] = pd.to_datetime(df['tpep_pickup_datetime']).dt.strftime('%Y-%m-%d %H')

In [12]:
df_corridas_por_dia = pd.pivot_table(df, values=['VendorID'],index='tpep_pickup_day' , aggfunc='count')
df_corridas_por_hora = pd.pivot_table(df, values=['VendorID'],index='tpep_pickup_hour' , aggfunc='count')

In [26]:
print(f'Máximo de corridas num dia: {df_corridas_por_dia.max()[0]}')
print(f'Média de corridas num dia: {df_corridas_por_dia.mean()[0]}')

Máximo de corridas num dia: 115471
Média de corridas num dia: 85704.55882352941


In [27]:
print(f'Máximo de corridas numa hora: {df_corridas_por_hora.max()[0]}')
print(f'Média de corridas numa hora: {df_corridas_por_hora.mean()[0]}')

Máximo de corridas numa hora: 8979
Média de corridas numa hora: 4186.716954022989


In [30]:
df = df[['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee']]

In [31]:
# padronizar os nomes das colunas 
df.columns = ['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'rate_code', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee']

In [32]:
df.shape[0]

2913955

In [ ]:
count = 0
virgula = ","
ponto_virgula = ";"
rate = 500 # taxa de inserção de linhas na tabela rides

while count < df.shape[0]:
   
   try:
      dml = f"""
         INSERT INTO rides ( vendor_id, pickup_datetime, dropoff_datetime, passenger_count, trip_distance, rate_code, store_and_fwd_flag, pickup_location_id, dropoff_location_id, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, airport_fee)
         VALUES """
      for c in range(count,count+rate):
         dml += f"""({df.iloc[c]["vendor_id"]}, timestamp '{df.iloc[c]["pickup_datetime"]}', timestamp '{df.iloc[c]["dropoff_datetime"]}', {df.iloc[c]["passenger_count"]}, {df.iloc[c]["trip_distance"]}, {df.iloc[c]["rate_code"]}, '{df.iloc[c]["store_and_fwd_flag"]}', {df.iloc[c]["pickup_location_id"]}, {df.iloc[c]["dropoff_location_id"]}, {df.iloc[c]["payment_type"]}, {df.iloc[c]["fare_amount"]}, {df.iloc[c]["extra"]}, {df.iloc[c]["mta_tax"]}, {df.iloc[c]["tip_amount"]}, {df.iloc[c]["tolls_amount"]}, {df.iloc[c]["improvement_surcharge"]}, {df.iloc[c]["total_amount"]}, {df.iloc[c]["congestion_surcharge"]}, {df.iloc[c]["airport_fee"]}){ponto_virgula if c == count+rate-1 else virgula}
         """
      cursor.execute(dml)
   except (Exception, psycopg2.Error) as error:
      print(error.pgerror)

   conn.commit()
   if count + rate > df.shape[0]:
      count += (df.shape[0] - count)
   else:
      count += rate
   print(count)

# teste iterando pela pasta 'data'

In [ ]:
for file in os.listdir('./data/'): 
   #  print(file)
   df = pd.read_parquet(file)